In [1]:
from asycn_model_tool import Qwen
import asyncio
from rag import RAG,VectorStore


In [2]:
'''
初始化模型、读取向量文档
'''


model=Qwen();
await model.start()

model_cache_dir = "./models"
save_dir = "./vector_store"


vector_store = VectorStore.load(
            save_dir=save_dir,
            embedding_model='shibing624/text2vec-base-chinese',
            model_cache_dir=model_cache_dir,
            device='cuda'
        )

rag = RAG(vector_store)



2025-03-09 15:32:20,259 - asycn_model_tool - INFO - 正在初始化 Qwen 模型...
2025-03-09 15:32:20,495 - accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-03-09 15:32:23,360 - asycn_model_tool - INFO - Qwen 模型初始化完成
2025-03-09 15:32:23,362 - asycn_model_tool - INFO - 批处理循环已启动
2025-03-09 15:32:23,365 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: shibing624/text2vec-base-chinese


模型已加载到 cuda 设备
向量存储已从 ./vector_store 加载，包含 23974 个文档


2025-03-09 15:32:24,133 - asycn_model_tool - INFO - 处理批次，批次大小: 1
2025-03-09 15:32:29,892 - asycn_model_tool - INFO - 批处理生成完成，处理时间: 5.76秒，批次大小: 1
2025-03-09 15:32:29,906 - asycn_model_tool - INFO - 处理批次，批次大小: 3
2025-03-09 15:32:30,269 - asycn_model_tool - ERROR - 批处理循环错误: CUDA out of memory. Tried to allocate 1.78 GiB. GPU 
Traceback (most recent call last):
  File "/root/autodl-tmp/asycn_model_tool.py", line 186, in _batch_process_loop
    results = await self._process_batch(batch)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/autodl-tmp/asycn_model_tool.py", line 241, in _process_batch
    generated_ids = self.model.generate(**batch_inputs, **self.generation_config)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/lib/python3.12/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/lib/python3.12/site-pac

In [3]:
'''
根据新闻文本查询数据建库
'''

text="这是一个在2022/1/1百家号发布的新闻。标题是匿名黑客Deric Lostutter面临16年监禁，而Steubenville强奸犯逍遥法外，正文是2016年9月14日 · 哼，我不管，Lostutter就是无罪不合理：曝光强奸犯的黑客可能比强奸犯本人面临更多监禁一条里根时代的网络安全法可以判定Deric Lostutter入狱16年。 一名黑客发现两个男人强暴一个无意识的16岁少女 …，是属于类别的新闻。请你告诉我是否是谣言。"


texts = [text, text, text, text]
images = ["blank.jpg", "blank.jpg", "blank.jpg", "blank.jpg"]  


for a in texts:
    a=rag.generate(a)+a
    print(a)

tasks = [model.predict(text, img) for text, img in zip(texts, images)]

results = await asyncio.gather(*tasks)

# 处理结果
for i, result in enumerate(results):
    print(f"任务 {i+1} 结果: {result}")
    



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

这是我所查到的文档:
文档 1 (相似度: 256.1937):
【user】: 这是一个在2022/1/1百家号发布的新闻。标题是匿名黑客Deric Lostutter面临16年监禁，而Steubenville强奸犯逍遥法外，正文是2016年9月14日 · 哼，我不管，Lostutter就是无罪不合理：曝光强奸犯的黑客可能比强奸犯本人面临更多监禁一条里根时代的网络安全法可以判定Deric Lostutter入狱16年。 一名黑客发现两个男人强暴一个无意识的16岁少女 …，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言



文档 2 (相似度: 201.9139):
【user】: 这是一个在2020/1/16Taiwan FactCheck Center发布的新闻。标题是反分裂法／反渗透法 1/15生效，选举的PO文不分蓝绿，请全部删掉以免惹事生非。1/16就会开始执行，正文是无内容，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言



文档 3 (相似度: 199.7449):
【user】: 这是一个在2017/10/26mygopen发布的新闻。标题是注意!!帐号姓名:崔瑞恩？骇客？要小心一封为「来抽支签！」的档案？谣言别再传了，正文是无内容，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言

这是一个在2022/1/1百家号发布的新闻。标题是匿名黑客Deric Lostutter面临16年监禁，而Steubenville强奸犯逍遥法外，正文是2016年9月14日 · 哼，我不管，Lostutter就是无罪不合理：曝光强奸犯的黑客可能比强奸犯本人面临更多监禁一条里根时代的网络安全法可以判定Deric Lostutter入狱16年。 一名黑客发现两个男人强暴一个无意识的16岁少女 …，是属于类别的新闻。请你告诉我是否是谣言。


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

这是我所查到的文档:
文档 1 (相似度: 256.1937):
【user】: 这是一个在2022/1/1百家号发布的新闻。标题是匿名黑客Deric Lostutter面临16年监禁，而Steubenville强奸犯逍遥法外，正文是2016年9月14日 · 哼，我不管，Lostutter就是无罪不合理：曝光强奸犯的黑客可能比强奸犯本人面临更多监禁一条里根时代的网络安全法可以判定Deric Lostutter入狱16年。 一名黑客发现两个男人强暴一个无意识的16岁少女 …，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言



文档 2 (相似度: 201.9139):
【user】: 这是一个在2020/1/16Taiwan FactCheck Center发布的新闻。标题是反分裂法／反渗透法 1/15生效，选举的PO文不分蓝绿，请全部删掉以免惹事生非。1/16就会开始执行，正文是无内容，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言



文档 3 (相似度: 199.7449):
【user】: 这是一个在2017/10/26mygopen发布的新闻。标题是注意!!帐号姓名:崔瑞恩？骇客？要小心一封为「来抽支签！」的档案？谣言别再传了，正文是无内容，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言

这是一个在2022/1/1百家号发布的新闻。标题是匿名黑客Deric Lostutter面临16年监禁，而Steubenville强奸犯逍遥法外，正文是2016年9月14日 · 哼，我不管，Lostutter就是无罪不合理：曝光强奸犯的黑客可能比强奸犯本人面临更多监禁一条里根时代的网络安全法可以判定Deric Lostutter入狱16年。 一名黑客发现两个男人强暴一个无意识的16岁少女 …，是属于类别的新闻。请你告诉我是否是谣言。


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

这是我所查到的文档:
文档 1 (相似度: 256.1937):
【user】: 这是一个在2022/1/1百家号发布的新闻。标题是匿名黑客Deric Lostutter面临16年监禁，而Steubenville强奸犯逍遥法外，正文是2016年9月14日 · 哼，我不管，Lostutter就是无罪不合理：曝光强奸犯的黑客可能比强奸犯本人面临更多监禁一条里根时代的网络安全法可以判定Deric Lostutter入狱16年。 一名黑客发现两个男人强暴一个无意识的16岁少女 …，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言



文档 2 (相似度: 201.9139):
【user】: 这是一个在2020/1/16Taiwan FactCheck Center发布的新闻。标题是反分裂法／反渗透法 1/15生效，选举的PO文不分蓝绿，请全部删掉以免惹事生非。1/16就会开始执行，正文是无内容，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言



文档 3 (相似度: 199.7449):
【user】: 这是一个在2017/10/26mygopen发布的新闻。标题是注意!!帐号姓名:崔瑞恩？骇客？要小心一封为「来抽支签！」的档案？谣言别再传了，正文是无内容，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言

这是一个在2022/1/1百家号发布的新闻。标题是匿名黑客Deric Lostutter面临16年监禁，而Steubenville强奸犯逍遥法外，正文是2016年9月14日 · 哼，我不管，Lostutter就是无罪不合理：曝光强奸犯的黑客可能比强奸犯本人面临更多监禁一条里根时代的网络安全法可以判定Deric Lostutter入狱16年。 一名黑客发现两个男人强暴一个无意识的16岁少女 …，是属于类别的新闻。请你告诉我是否是谣言。


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

这是我所查到的文档:
文档 1 (相似度: 256.1937):
【user】: 这是一个在2022/1/1百家号发布的新闻。标题是匿名黑客Deric Lostutter面临16年监禁，而Steubenville强奸犯逍遥法外，正文是2016年9月14日 · 哼，我不管，Lostutter就是无罪不合理：曝光强奸犯的黑客可能比强奸犯本人面临更多监禁一条里根时代的网络安全法可以判定Deric Lostutter入狱16年。 一名黑客发现两个男人强暴一个无意识的16岁少女 …，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言



文档 2 (相似度: 201.9139):
【user】: 这是一个在2020/1/16Taiwan FactCheck Center发布的新闻。标题是反分裂法／反渗透法 1/15生效，选举的PO文不分蓝绿，请全部删掉以免惹事生非。1/16就会开始执行，正文是无内容，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言



文档 3 (相似度: 199.7449):
【user】: 这是一个在2017/10/26mygopen发布的新闻。标题是注意!!帐号姓名:崔瑞恩？骇客？要小心一封为「来抽支签！」的档案？谣言别再传了，正文是无内容，是属于类别的新闻。[图片: images/blank.jpg]请你告诉我是否是谣言。

【assistant】: 谣言

这是一个在2022/1/1百家号发布的新闻。标题是匿名黑客Deric Lostutter面临16年监禁，而Steubenville强奸犯逍遥法外，正文是2016年9月14日 · 哼，我不管，Lostutter就是无罪不合理：曝光强奸犯的黑客可能比强奸犯本人面临更多监禁一条里根时代的网络安全法可以判定Deric Lostutter入狱16年。 一名黑客发现两个男人强暴一个无意识的16岁少女 …，是属于类别的新闻。请你告诉我是否是谣言。


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.78 GiB. GPU 